# Overview

This example shows how to extract a time series of images from Earth Engine, annotate them, and save them as a video.

# Initialize Earth Engine

We start by importing the [Earth Engine Python API](https://pypi.org/project/earthengine-api/) module.

In [ ]:
import ee

The following command initializes the Earth Engine Python API.

In [ ]:
ee.Initialize()

If the cell output displays an error about needing to authenticate, open up a  JupyterLab terminal tab and run the command: `earthengine authenticate`

# Building an image collection

In [ ]:
import datetime
import ipywidgets as widgets
import ipyleaflet  # an interactive mapping "widget"
from sidecar import Sidecar

## Define User Interface Elements

In [ ]:
roi_dimension = widgets.IntSlider(
    value=1e4,
    min=1e2,
    max=1e5,
    description='ROI Size (m):',
    continuous_update=False,
)

In [ ]:
# Define the map.
map1 = ipyleaflet.Map(
    center=(0, -80.0),
    zoom=8,
    layout={'height':'300px'},
)
map1.add_control(ipyleaflet.LayersControl())

# Define and add a Marker pin to the map.
center_marker = ipyleaflet.Marker(
    name='ROI Selection Marker',
    location=map1.center
)
map1 += center_marker

# Define a ROI layer group and add it to the map.
# roi_layer = ipyleaflet.Polygon(
#     name='ROI Polygon',
#     locations=[(0, -80), (0, -81), (1, -81)],
#     color="green",
#     fill_color="green"
# )
# map1 += roi_layer
roi_layer_group = ipyleaflet.LayerGroup(layers=())
map1 += roi_layer_group

In [ ]:
start_datepicker = widgets.DatePicker(
    description='Start Date',
    disabled=False,
    value=datetime.datetime(2017, 8, 20)
)
end_datepicker = widgets.DatePicker(
    description='End Date',
    disabled=False,
    value=datetime.datetime(2017, 9, 20)
)

In [ ]:
out = widgets.Output()

## Display the UI Elements

In [ ]:
# Layout the UI elements.
panel = widgets.VBox([
    map1,
    start_datepicker,
    end_datepicker,
    roi_dimension,
    out
])

# Display the UI elements in a side panel.
sc = Sidecar(title='UI Panel')
with sc:
    display(panel)

## Define interactions

In [ ]:
# Define a helper function to swap the coordinate ordering.

def swap_coordinate_xy_for_location(coord):
    return (coord[1],coord[0]) 
def swap_coordinate_xy_for_list(coord_list):
    return [swap_coordinate_xy_for_location(coord) for coord in coord_list]

def update_roi_layer(map_reference):
#     print('DEBUG starting update_roi_layer')
    
    center_marker_xy = swap_coordinate_xy_for_location(center_marker.location)
    centroid = ee.Geometry.Point(center_marker_xy)
    buffered = centroid.buffer(roi_dimension.value).bounds()
    coord_list_xy = buffered.getInfo()['coordinates'][0]
    coord_list_yx = swap_coordinate_xy_for_list(coord_list_xy)
    
    roi_layer = ipyleaflet.Polygon(
        name='TEST update ROI Polygon',
        locations=coord_list_yx,
        weight=3,
        color='#F00',
        opacity=0.8,
        fill_opacity=0.1,
        fill_color='#F00'
    )
    roi_layer_group.clear_layers()
    roi_layer_group.add_layer(roi_layer)
    
#     print('DEBUG ending update_roi_layer')

In [ ]:
# Define the actions performed when the marker moves.
def center_marker_on_move(change):
    with out:
#         print('DEBUG starting center_marker_on_move')
#         print(change['new'])
        update_roi_layer(map1)
center_marker.unobserve_all()
center_marker.observe(center_marker_on_move, names='location')

In [ ]:
# Define the actions performed when the ROI size is changed.
def roi_dimension_on_change(change):
    with out:
#         print('DEBUG starting roi_dimension_on_change')
#         print(change['new'])
        update_roi_layer(map1)
roi_dimension.unobserve_all()
roi_dimension.observe(roi_dimension_on_change, names='value')

# Testing out a series

This section will demonstrate outputing an image time series for a hardcode location, time interval, and image collection.

In [ ]:
roi = {'type': 'Polygon',
  'coordinates': [[[-62.091508, 80.942168],
    [-62.027586, 80.774083],
    [-59.974902, 80.780053],
    [-59.992905, 80.945812],
    [-62.091508, 80.942168]]]}

# roi = {'type': 'Polygon',
#   'coordinates': [[[-61.569142, 80.917193],
#     [-60.891938, 80.959731],
#     [-61.01253, 80.832927],
#     [-61.569142, 80.917193]]]}

In [ ]:
start_date = ee.Date('2018-01-01')
end_date = ee.Date('2018-02-01')
collection = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
      .filter(ee.Filter.eq('relativeOrbitNumber_start', 90.0))
      .filter(ee.Filter.eq('sliceNumber', 16.0))
      .filterDate(start_date, end_date)
      .filterBounds(roi)
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV'))
      .select('HV')
)
print(collection.size().getInfo())

In [ ]:
MAX_ELEMENTS=100
images = collection.toList(MAX_ELEMENTS).getInfo()

In [ ]:
[img['id'] for img in images]

In [ ]:
import os
output_directory = 'output'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

In [ ]:
import datetime
from IPython.display import Image
import imageio
import pprint
import PIL.Image
import PIL.ImageDraw
import PIL.ImageFont
import requests
import numpy

pp = pprint.PrettyPrinter(indent=4)

sc = Sidecar(title='Images!')
with sc:
    image_no = 0
    
    for img in images:
        image_no += 1
        id = img['id']
        print(id)

        image_time_ms = img['properties']['system:time_start']
        start_time = datetime.datetime.fromtimestamp(image_time_ms/1000.0)

        sample = ee.Image(id)
        url = sample.getThumbUrl({
            'bands': 'HH',
            'min':-20,
            'max':0,
            'region':roi
        })
        

        # Create a base image, from the Earth Engine Thumbnail URL.
        base_ee = PIL.Image.open(requests.get(url, stream=True).raw).convert('RGBA')

        # Create a background image.
        background = PIL.Image.new('RGBA', base_ee.size, (12,12,12,255))
        
        # Burn the EE data onto the backgound.
        base = PIL.Image.alpha_composite(background, base_ee)
        
        # make a blank image for the text, initialized to transparent text color
        txt = PIL.Image.new('RGBA', base_ee.size, (255,255,255,0))
        
        # Get a font
        #fnt = PIL.ImageFont.truetype('Pillow/Tests/fonts/FreeMono.ttf', 30)
        fnt = PIL.ImageFont.truetype('Pillow/Tests/fonts/DejaVuSans.ttf', 30)
        
        # get a drawing context
        d = PIL.ImageDraw.Draw(txt)
        # Draw text. For date formatting codes see:
        # https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
        date_string = '{:%Y-%m-%d %H:%M:%S}'.format(start_time)
        d.text((10,10), date_string, font=fnt, fill=(255,0,0,255))
        # Composite the image and text annotation.
        out = PIL.Image.alpha_composite(base, txt)
        
        outfile = '{0}/img{1:03d}.png'.format(output_directory, image_no)
        out.save(outfile, format='PNG')

        display(out)

## Create an animated GIF.

In [ ]:
import glob

gif_images = []
filenames = sorted(glob.glob('output/img???.png'))
for filename in filenames:
    gif_images.append(imageio.imread(filename))
imageio.mimsave('output/movie.gif', gif_images, 'GIF', duration=0.5)

## Create an MPEG video.

In [ ]:
%%bash  
ffmpeg -r 1 \
    -i output/img%03d.png \
    -vcodec mpeg4 \
    -y output/out.mp4

# Experiment with BQPlot date range selection

In [ ]:
import bqplot
import pandas as pd
import numpy as np

In [ ]:
symbol = 'Security 1'

price_data = pd.DataFrame(
    np.cumsum(np.random.randn(150, 2).dot([[0.5, 0.4], [0.4, 1.0]]), axis=0) + 100,
    columns=['Security 1', 'Security 2'],
    index=pd.date_range(start='01-01-2007', periods=150))

dates_actual = price_data.index.values
prices = price_data[symbol].values

In [ ]:
## First we define a Figure
dt_x_fast = bqplot.DateScale()
lin_y = bqplot.LinearScale()

x_ax = bqplot.Axis(
    label='Index', scale=dt_x_fast)
x_ay = bqplot.Axis(label=(symbol + ' Price'), scale=lin_y, orientation='vertical')
lc = bqplot.Lines(x=dates_actual, y=prices, scales={'x': dt_x_fast, 'y': lin_y}, colors=['orange'])


In [ ]:
## Next we define the type of selector we would like
intsel_fast = bqplot.interacts.FastIntervalSelector(scale=dt_x_fast, marks=[lc, ])

In [ ]:
## We use the HTML widget to see the value of what we are selecting and modify it when an interaction is performed
## on the selector
db_fast = ipywidgets.HTML()
db_fast.value = 'The selected period is ' + str(intsel_fast.selected)

fig_fast_intsel = bqplot.Figure(
    marks=[lc, ],
    axes=[x_ax, x_ay],
    title='Fast Interval Selector Example',
    interaction=intsel_fast #This is where we assign the interaction to this particular Figure
)

ipywidgets.VBox([db_fast, fig_fast_intsel])